This code is to perform facial detection on the training data

### Import Modules

In [1]:
import numpy as np
import time
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image

### Define File Paths

In [2]:
# i think r is raw, f is format (for string file paths)
INPUT_FOLDER = r'D:\GABY\UPH\TA2\Code2\folder-training'
OUTPUT_FOLDER = r'D:\GABY\UPH\TA2\Code2\face-detection-result'
OUTPUT_FOLDER_CLEAN = r'D:\GABY\UPH\TA2\Code2\face-detection-result-clean'
LABELS_FILE=r'C:\Users\asus\Darknet\darknet-master\data\obj.names'
CONFIG_FILE=r'C:\Users\asus\Darknet\darknet-master\cfg\yolov3-custom2.cfg'
WEIGHTS_FILE=r'C:\Users\asus\Darknet\darknet-master\backup\yolov3-custom2_last.weights'

### Initialize YOLO

In [3]:
#try modify this?
CONFIDENCE_THRESHOLD=0.05

LABELS = open(LABELS_FILE).read().strip().split("\n")

net = cv2.dnn.readNetFromDarknet(CONFIG_FILE, WEIGHTS_FILE)


# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

start = time.time()
end = time.time()


print("[INFO] YOLO took {:.6f} seconds".format(end - start))

[INFO] YOLO took 0.015113 seconds


In [6]:
#find number of images in file
list = os.listdir(INPUT_FOLDER) # dir is your directory path
number_files = len(list)
number_files

276

### Detection

In [14]:
num_res = 0
for iteration in range(number_files):
    
    image = cv2.imread(f'{INPUT_FOLDER}/picture ({str(iteration+1)}).JPG')
    (H, W) = image.shape[:2]
    
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
    swapRB=True, crop=False)
    net.setInput(blob)
    
    layerOutputs = net.forward(ln)
    
    # initialize our lists of detected bounding boxes, confidences, and
    # class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []
    
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            scores = detection[5:] #get class probability
            classID = np.argmax(scores) #get index of max class probability
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > CONFIDENCE_THRESHOLD:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression to suppress weak, overlapping bounding
    # boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD,
        CONFIDENCE_THRESHOLD)

    result = 0
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            result=result+1

            #draw bounding boxes, save as new image file
            cv2.rectangle(image, (x, y), (x + w, y + h), color=(255,0,0), thickness=2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, color=(255,0,0), thickness=3)
            roi = image[y:y+h, x:x+w]
            #path = os.path.join(OUTPUT_IMAGE, str(result))
            cv2.imwrite(f'{OUTPUT_FOLDER}/{str(iteration+1)}--{str(result)}.jpg', roi)
            cv2.imwrite(f'{OUTPUT_FOLDER_CLEAN}/{str(num_res+1)}.jpg', roi)
            num_res = num_res+1